### Optuna

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time
import optuna


#xgboost
from xgboost import XGBClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score,roc_curve
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_diab3 import * #(NOVO atualizações)
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()

ModuleNotFoundError: No module named 'setup_notebook'

## 1. Load Data & Pipeline

In [2]:
BASE = Path.cwd().parent

PP3 = joblib.load(BASE/'src'/'preprocess_diabetes_v3.joblib')['pipeline']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
mtd_scoring='roc_auc'

## 2.Baseline

In [14]:
def  best_threshold(y_test,yprob):
    # Threshold search
    thresholds = np.linspace(0.3, 0.7, 41)
    best_val = 0.5
    max_acc = 0
    for t in thresholds:
        acc = accuracy_score(y_test, yprob > t)
        if acc > max_acc:
            max_acc = acc
            best_val = t
    return best_val,max_acc

def print_hiper(search_1_best_params):
    
    print("🎯 Melhores Hiperparâmetros")
    print("-" * 50)
    for param, value in search_1_best_params.items():
        param_name = param.replace('model__', '').replace('_', ' ').title()
        print(f"• {param_name:<25} : {value}")
    print("-" * 50)

#=====================================================================
# model Baseline
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
model_base = XGBClassifier(objective='binary:logistic',eval_metric='logloss',enable_categorical=True, tree_method="hist" , random_state=42)
pipe_base  = pipe_models(model_base, PP3)

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipe_base.fit(X_train, y_train)
y_probs0 = pipe_base.predict_proba(X_test)[:, 1]

threshold_0,max_acc_0=best_threshold(y_test,y_probs0)
auc_score_0 = roc_auc_score(y_test, y_probs0)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_0:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_0:.4f}")
print(f"⭐ AUC Score                : {auc_score_0:.4f}")
print(f"{'='*70}")
print_hiper(model_base.get_params())

print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo iniciado em: 17:12:46
🎯 Melhor Threshold         : 0.510
📈 Melhor Acurácia de Teste : 0.6805
⭐ AUC Score                : 0.7209
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Objective                 : binary:logistic
• Base Score                : None
• Booster                   : None
• Callbacks                 : None
• Colsample Bylevel         : None
• Colsample Bynode          : None
• Colsample Bytree          : None
• Device                    : None
• Early Stopping Rounds     : None
• Enable Categorical        : False
• Eval Metric               : logloss
• Feature Types             : None
• Gamma                     : None
• Grow Policy               : None
• Importance Type           : None
• Interaction Constraints   : None
• Learning Rate             : None
• Max Bin                   : None
• Max Cat Threshold         : None
• Max Cat To Onehot         : None
• Max Delta Step            : None
• Max Depth                 : No

## 3.Buscas por hiperparamentros


In [15]:
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))

#função objeto(optuna)
def objective(trial):
    
    param_dist_1 = {
    # 1. Controle de Aprendizado
    'model__n_estimators':  trial.suggest_int("n_estimators",100, 3000), 
    'model__learning_rate': trial.suggest_float("learning_rate",0.003,0.3,log=True),
    
    # 2. Complexidade e Profundidade
    'model__max_depth': trial.suggest_int("max_depth", 3, 13),
    'model__min_child_weight':trial.suggest_int("min_child_weight", 1, 20),
    
    # 3. Robustez contra Overfitting (Stochastic Gradient Boosting)
    'model__subsample':  trial.suggest_float("subsample", 0.5, 1.0),
    'model__colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
    
    # 4. Regularização (Penalidades)
    'model__gamma': trial.suggest_float("gamma", 0.0, 5.0),
    'model__reg_alpha': trial.suggest_float("reg_alpha", 0.0, 5.0),
    'model__reg_lambda': trial.suggest_float("reg_lambda", 0.1, 10.0),}

#     param_dist_refino = {
#     # Foco na zona de alta performance
#     'model__n_estimators': trial.suggest_int("n_estimators", 1800, 2800),
#     'model__learning_rate': trial.suggest_float("learning_rate", 0.008, 0.04), 
    
#     # Estabilizando a arquitetura das árvores
#     'model__max_depth': trial.suggest_int("max_depth", 7, 10),
#     'model__min_child_weight': trial.suggest_int("min_child_weight", 15, 40), # Aumentei para filtrar ruído
    
#     # Mantendo a diversidade que funcionou
#     'model__subsample': trial.suggest_float("subsample", 0.75, 0.95),
#     'model__colsample_bytree': trial.suggest_float("colsample_bytree", 0.45, 0.55),
    
#     # Apertando a regularização para buscar o 0.73
#     'model__gamma': trial.suggest_float("gamma", 1.5, 4.0),
#     'model__reg_alpha': trial.suggest_float("reg_alpha", 3.0, 7.0),
#     'model__reg_lambda': trial.suggest_float("reg_lambda", 2.0, 6.0),
# }

    pipe_base.set_params(**param_dist_1)
    cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(pipe_base,X_train, y_train,scoring="roc_auc",cv=cv_s,n_jobs=-1
    ).mean()

    return score
    
start = time.time()    
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42,multivariate=True))

numero_itera=3
study.optimize(objective,n_trials=numero_itera,show_progress_bar=True)

best_pipe = pipe_base.set_params(
    **{f"model__{k}": v for k, v in study.best_params.items()})


end = time.time()

best_pipe.fit(X_train, y_train)

y_probs1 = best_pipe.predict_proba(X_test)[:, 1]

threshold_1, max_acc_1 = best_threshold(y_test, y_probs1)
auc_score_1 = roc_auc_score(y_test, y_probs1)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_1:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_1:.4f}")
print(f"⭐ AUC Score                : {auc_score_1:.4f}")
print(f"{'='*70}")

print_hiper(study.best_params)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


[I 2026-02-01 17:12:57,577] A new study created in memory with name: no-name-e83e7006-8884-45c5-9aa2-7a7b9ea12a50



#Processo iniciado em: 17:12:57


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-02-01 17:14:50,305] Trial 0 finished with value: 0.7244050407067121 and parameters: {'n_estimators': 2174, 'learning_rate': 0.03842285780511732, 'max_depth': 9, 'min_child_weight': 30, 'subsample': 0.7812037280884873, 'colsample_bytree': 0.4655994520336203, 'gamma': 1.6452090304204987, 'reg_alpha': 6.46470458309974, 'reg_lambda': 4.404460046972835}. Best is trial 0 with value: 0.7244050407067121.
[I 2026-02-01 17:17:57,977] Trial 1 finished with value: 0.7260105673141085 and parameters: {'n_estimators': 2508, 'learning_rate': 0.008658703817465679, 'max_depth': 10, 'min_child_weight': 36, 'subsample': 0.7924678221356553, 'colsample_bytree': 0.4681824967207101, 'gamma': 1.9585112746335844, 'reg_alpha': 4.216968971838151, 'reg_lambda': 4.099025726528952}. Best is trial 1 with value: 0.7260105673141085.
[I 2026-02-01 17:19:47,818] Trial 2 finished with value: 0.7257599736363685 and parameters: {'n_estimators': 2232, 'learning_rate': 0.01731933248633734, 'max_depth': 9, 'min_child_w

In [6]:
end_final = time.time()
print(f"Tempo final: {(end_final-start_inicial)/60:.2f} min")

Tempo final: 119.76 min


In [9]:
#Salvar Hiperparametros joblib

# joblib.dump(search_1.best_estimator_, 'modelo_XGB_final_randsearch.'+mtd_scoring+'_v1.2.joblib')
# joblib.dump(search_2.best_estimator_, 'modelo_XGB_final_refine.'+mtd_scoring+'_v1.2.joblib')

['modelo_XGB_final_refine.roc_auc_v1.2.joblib']